In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [60]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
import pickle
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed



In [3]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]


In [4]:
feed = DataFeed()

In [5]:
chain = feed.get_fut_chain('US.CL')

In [6]:
futures = chain.get_all()

In [66]:
fut = futures[0]

In [67]:
print('Contract: {0} SqlID: {1}'.format(fut, fut.contract_info.extra('sqlid')))

Contract: US.F.CL.N83.830622 SqlID: 2.0


In [20]:
# Storing futures
mongo_collection = mongo_db['tmp_bardata']
mongo_collection.create_index([('idcontract', pymongo.ASCENDING), ('datetime', pymongo.ASCENDING)])

'idcontract_1_datetime_1'

In [62]:
quotes_collection = mongo_db['quotes_intraday']
quotes_collection.create_index([('tckr', pymongo.ASCENDING), ('dt',pymongo.ASCENDING)], unique=True)

'tckr_1_dt_1'

In [71]:
for fut in tqdm_notebook(futures):
    data = list(mongo_collection.find({'idcontract': fut.contract_info.extra('sqlid')}))
    if len(data) == 0:
        print("Empty contract series for {0} ... skipping".format(fut))
        continue
    df = pd.DataFrame(data)
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    df.rename(columns={'datetime': 'dt', 'open': 'o', 'high': 'h', 'low':'l', 'close': 'c', 'volume': 'v'}, inplace=True)
    df.set_index('dt', inplace=True)
    df.index = df.index.tz_localize(fut.instrument_info.timezone).tz_convert('UTC')
    
    for idx_dt, df_value in df.groupby(by=df.index.date):
        dt = datetime.combine(idx_dt, time(0,0,0))
        rec = {
            'dt': dt,
            'tckr': fut.ticker,
            'ohlc': pickle.dumps(df_value)
            }
        quotes_collection.replace_one({'dt': dt, 'tckr': fut.ticker}, rec, upsert=True)    


Empty contract series for US.F.CL.N83.830622 ... skipping
Empty contract series for US.F.CL.Q83.830720 ... skipping
Empty contract series for US.F.CL.U83.830822 ... skipping
Empty contract series for US.F.CL.V83.830921 ... skipping
Empty contract series for US.F.CL.X83.831020 ... skipping
Empty contract series for US.F.CL.F84.831221 ... skipping
Empty contract series for US.F.CL.G84.840120 ... skipping
Empty contract series for US.F.CL.H84.840222 ... skipping
Empty contract series for US.F.CL.J84.840321 ... skipping
Empty contract series for US.F.CL.K84.840419 ... skipping
Empty contract series for US.F.CL.M84.840522 ... skipping
Empty contract series for US.F.CL.N84.840620 ... skipping
Empty contract series for US.F.CL.Q84.840720 ... skipping
Empty contract series for US.F.CL.U84.840822 ... skipping
Empty contract series for US.F.CL.V84.840920 ... skipping
Empty contract series for US.F.CL.X84.841022 ... skipping
Empty contract series for US.F.CL.F85.841220 ... skipping
Empty contract